In [1]:
import sys
sys.path.append("../input/tez-lib")
sys.path.append("../input/timmmaster")
sys.path.append("../input/kyosukepet")

In [2]:
import tez
import albumentations
import pandas as pd
import numpy as np
import timm
from sklearn import metrics
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

import glob
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import models, transforms

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
test_df=pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
test_df.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1
3,80bc3ccafcc51b66303c2c263aa38486,1,0,1,0,0,0,0,0,0,0,1,0
4,8f49844c382931444e68dffbe20228f4,1,1,1,0,1,1,0,1,0,1,1,0


In [5]:
test_df["Path"]=["../input/petfinder-pawpularity-score"+"/test/"+x+".jpg" for x in test_df["Id"]]

In [6]:
test_df["Pawpularity"]=[0 for _ in range(len(test_df))]
test_df

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Path,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1,../input/petfinder-pawpularity-score/test/4128...,0
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0,../input/petfinder-pawpularity-score/test/43a2...,0
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1,../input/petfinder-pawpularity-score/test/4e42...,0
3,80bc3ccafcc51b66303c2c263aa38486,1,0,1,0,0,0,0,0,0,0,1,0,../input/petfinder-pawpularity-score/test/80bc...,0
4,8f49844c382931444e68dffbe20228f4,1,1,1,0,1,1,0,1,0,1,1,0,../input/petfinder-pawpularity-score/test/8f49...,0
5,b03f7041962238a7c9d6537e22f9b017,0,0,1,1,1,1,1,1,1,0,1,0,../input/petfinder-pawpularity-score/test/b03f...,0
6,c978013571258ed6d4637f6e8cc9d6a3,1,0,0,0,1,1,0,1,0,1,1,1,../input/petfinder-pawpularity-score/test/c978...,0
7,e0de453c1bffc20c22b072b34b54e50f,1,0,1,0,0,0,0,0,1,0,0,1,../input/petfinder-pawpularity-score/test/e0de...,0


In [7]:
if torch.cuda.is_available():
    device="cuda:0"
else:
    device="cpu"

In [8]:
class ImageTransform():
    def __init__(self, resize, mean,std):
        self.data_transform={
            "train":transforms.Compose([
            transforms.RandomResizedCrop(
            resize,scale=(0.5,1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean,std)
        ]),
            "val": transforms.Compose([
                transforms.Resize(resize),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean,std)
            ])
        }
        
    def __call__(self,img,phase="train"):
        return self.data_transform[phase](img)   

In [9]:
class PawpularDataset(Dataset):
    def __init__(self, df,dense_features,transform=None, phase="val"):
        self.df = df
        self.transform=transform
        self.phase=phase #train or val
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        image_path=self.df["Path"].iloc[index]
        img=Image.open(image_path)
        img_transformed=self.transform(img,self.phase) #torch.Size([3,224,224])
        
        dense_feature=self.df[dense_features].loc[index,:].values
        
        target=self.df["Pawpularity"].iloc[index]
    
        return img_transformed,dense_feature, target

In [10]:
class VGG16Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.model=models.vgg16(pretrained=False)
        self.model.classifier[6]=nn.Linear(in_features=4096, out_features=2048)
        self.dropout1=nn.Dropout(0.5)
        self.dense1=nn.Linear(2048,256)
        self.dropout2=nn.Dropout(0.5)
        self.dense2=nn.Linear(256+12,32)
        self.dense3=nn.Linear(32,1)
        
    def forward(self,image,features,targets=None):
        x=self.model(image)
        x=self.dropout1(x)
        x=self.dense1(x)
        x=self.dropout2(x)
        x=torch.cat([x,features],dim=1)
        x=self.dense2(x)
        x=self.dense3(x)
        
        return x

In [11]:
class Resnet50Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.model=models.resnet50(pretrained=False)
        self.fc=nn.Linear(1000,128)
        self.dense1=nn.Linear(128,64)
        self.dropout1=nn.Dropout(0.5)
        self.dense2=nn.Linear(64+12,32)
        self.dropout2=nn.Dropout(0.5)
        self.dense3=nn.Linear(32,1)
        
    def forward(self,image,features,targets=None):
        x=self.model(image)
        x=self.fc(x)
        x=self.dense1(x)
        x=self.dropout1(x)
        x=torch.cat([x,features],dim=1)
        x=self.dense2(x)
        x=self.dropout2(x)
        x=self.dense3(x)
        
        return x

In [12]:
batch_size=16
resize=224
mean=(0.485, 0.456, 0.406)
std=(0.229, 0.224, 0.225)


In [13]:
#seedを設定
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed(123)

In [14]:
super_final_predictions=[]
for fold_ in range(5):
    #Intialize the model
    model1= Resnet50Model()
    model2 = VGG16Model()
    
    #load model from path
    model1.load_state_dict(torch.load('../input/kyosukepet/resnet50_weights_10.pth'))
    model2.load_state_dict(torch.load('../input/kyosukepet/vgg16_weights_10.pth'))
    
    model1.to(device)
    model2.to(device)
    
    model1.eval()
    model2.eval()
    
    #create list of dense features
    dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]
        
    #Create dataset
    test_dataset=PawpularDataset(test_df,dense_features,transform=ImageTransform(resize,mean,std),phase="val")
    
    test_dataloader=DataLoader(test_dataset,batch_size=batch_size,shuffle=False)
    
    pred1=[]
    pred2=[]
    for images,features,_ in tqdm(test_dataloader):
        images=torch.tensor(images, dtype=torch.float)
        features=torch.tensor(features,dtype=torch.float)
                
        images=images.to(device)
        features=features.to(device)
        
        outputs1=model1(images,features)
        outputs2=model2(images,features)
        
        pred1 += [output1 for output1 in outputs1]
        pred2 += [output2 for output2 in outputs2]
        
    test_predictions = list(0.45*np.array(pred1)+0.45*np.array(pred2))
    
    #put predictions in final_test_predictions list
    final_test_predictions=[]
    for preds in tqdm(test_predictions):
        final_test_predictions.extend(preds)
         
    final_test_predictions=[x.item() for x in final_test_predictions]
    super_final_predictions.append(final_test_predictions)
    
#add the list to super final predictions list
super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)        
#take average over all the folds

100%|██████████| 8/8 [00:00<00:00, 1584.62it/s]


In [15]:
test_df['Pawpularity']=super_final_predictions
test_df=test_df[['Id','Pawpularity']]
test_df.to_csv("submission.csv", index=False)
test_df

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,33.358948
1,43a2262d7738e3d420d453815151079e,32.526752
2,4e429cead1848a298432a0acad014c9d,33.447830
3,80bc3ccafcc51b66303c2c263aa38486,32.255741
4,8f49844c382931444e68dffbe20228f4,33.726196
5,b03f7041962238a7c9d6537e22f9b017,33.458778
6,c978013571258ed6d4637f6e8cc9d6a3,33.149063
7,e0de453c1bffc20c22b072b34b54e50f,32.749748
